<a href="https://colab.research.google.com/github/andervies/divic-corp-machine-learning-course/blob/main/assignment33/Recurrent_Neural_Network_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem One: Simple Forward propagation implementation of RNN

In [10]:
import numpy as np

# Forward pass function
def forward_pass(inputs, hidden_state):
    for sequence in range(inputs.shape[1]):
        hidden_state = np.tanh(inputs[:, sequence, :] @ weights_input + hidden_state @ weights_hidden + bias)
    return hidden_state

## Problem Two: Experiment of forward propagation with small sequence

In [11]:
# Input data
inputs = np.array([[[1, 2], [2, 3], [3, 4]]]) / 100
weights_input = np.array([[1, 3, 5, 7], [3, 5, 7, 8]]) / 100
weights_hidden = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]]) / 100
hidden_state = np.zeros((inputs.shape[0], weights_input.shape[1]))
bias = np.array([1, 1, 1, 1])

forward_pass(inputs, hidden_state)

array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])

## Problem Three (Advance assignment) Implementation of backpropagation

In [12]:
# Initializer class
class SimpleInitializer:
    def __init__(self, sigma):
        self.sigma = sigma

    def initialize_weights(self, input_size, output_size):
        return self.sigma * np.random.randn(input_size, output_size)

    def initialize_bias(self, output_size):
        return self.sigma * np.random.randn(1, output_size)


In [13]:
# SGD optimizer class
class SGD:
    def __init__(self, learning_rate):
        self.learning_rate = learning_rate

    def update(self, layer):
        layer.weights_input -= self.learning_rate * layer.grad_weights_input
        layer.weights_hidden -= self.learning_rate * layer.grad_weights_hidden
        layer.bias -= self.learning_rate * layer.grad_bias
        return layer



In [14]:
# Tanh activation class
class Tanh:
    def forward(self, inputs):
        self.inputs = inputs
        return np.tanh(self.inputs)

    def backward(self, grad_output):
        return grad_output * (1 - np.tanh(self.inputs) ** 2)


In [15]:
# Neural network classifier class
class ScratchDeepNeuralNetworkClassifier:
    def __init__(self):
        self.weights_input = np.array([[1, 3, 5, 7], [3, 5, 7, 8]]) / 100
        self.weights_hidden = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]]) / 100
        self.bias = np.array([1, 1, 1, 1])
        self.grad_bias = 0
        self.grad_weights_hidden = 0
        self.grad_weights_input = 0
        self.activator = Tanh()
        self.optimizer = SGD(learning_rate=0.001)
        self.hidden_history = []

    def forward(self, inputs):
        self.inputs = inputs
        batch_size = inputs.shape[0]
        num_sequences = inputs.shape[1]
        num_nodes = self.weights_input.shape[1]
        hidden_state = np.zeros((batch_size, num_nodes))

        for sequence in range(num_sequences):
            activation = inputs[:, sequence, :] @ self.weights_input + hidden_state @ self.weights_hidden + self.bias
            hidden_state = self.activator.forward(activation)
            self.hidden_history.append(hidden_state)

        return hidden_state

    def backward(self, grad_output):
        grad_activation = self.activator.backward(grad_output)

        for sequence in reversed(range(self.inputs.shape[1])):
            self.grad_bias += np.sum(grad_activation, axis=0)
            self.grad_weights_input += self.inputs[:, sequence, :].T @ grad_activation
            self.grad_weights_hidden += self.hidden_history[sequence].T @ grad_activation

            grad_input = grad_activation @ self.weights_input.T
            grad_output = grad_activation @ self.weights_hidden.T

        return grad_input, grad_output



In [16]:
# Testing the neural network classifier
test_classifier = ScratchDeepNeuralNetworkClassifier()
test_classifier.forward(inputs)


array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])